In [4]:
import numpy as np
import matplotlib as plt

In [5]:
k = 100 # 簇的数量
agent = 20 # 智能体的个数

In [6]:
dataset_path = r"D:\Documents\QG工作室\工作室考核\中期考核\数据集\s1.txt"
network_path = r"D:\Documents\QG工作室\工作室考核\中期考核\数据集\switch_network1.npz"

In [48]:
As = np.load(network_path)
As = [As['A1'], As['A2'], As['A3']]

In [11]:
Y = []
with open(dataset_path, 'r') as file:
    lines = file.readlines()
    for oneline in lines:
        oneline = oneline[:-1]
        oneline = oneline.split()
        Y.append([float(oneline[0]),float(oneline[1])])
Y = np.array(Y) # 原始数据集

In [12]:
m = Y.shape[0]  # 数据的总个数
mi = int(m/agent) # 每个智能体分到的数据量

In [17]:
Yis = []
idx = np.arange(m)    
np.random.shuffle(idx) # 打乱
for i in range(agent):
    Yis.append(Y[idx[i*mi:(i+1)*mi]]) # 每个智能体对应处理多少数据
Yis = np.array(Yis)

In [21]:
ci = [] # 存放所有的聚类中心
# k-eans++开始
# 确定所有的聚类中心
for i in range(agent):
    ci_temp = [] # 存放每个智能体数据的聚类中心
    first_idx = np.random.randint(0,mi)
    ci_temp.append(Yis[i][first_idx])
    for j in range(k-1):
        min_dis = [] # 每个点到最近簇的距离
        for one_y in Yis[i]:
            temp_dis = [] # 每个点到所有簇的距离
            for one_ci in ci_temp:
                temp_dis.append(np.linalg.norm(one_y-one_ci))
            min_dis.append(np.min(temp_dis))
        min_dis = np.array(min_dis)
        next_idx = np.argmax(min_dis)
        ci_temp.append(Yis[i][next_idx])
    ci.append(np.array(ci_temp))
ci

[array([[367641., 556309.],
        [869167., 105039.],
        [854628., 773470.],
        [297177.,  88467.],
        [187894., 871219.],
        [657450., 365008.],
        [153903., 353738.],
        [496658., 816784.],
        [900991., 497149.],
        [115287., 602637.],
        [606738., 601356.],
        [518892., 165875.],
        [683048., 937776.],
        [370252., 347498.],
        [817119., 336333.],
        [927170., 250593.],
        [335121., 214149.],
        [202627., 179376.],
        [264798., 767716.],
        [612648., 772280.],
        [813561., 583156.],
        [467535., 416014.],
        [296892., 901182.],
        [216209., 437830.],
        [438437., 730300.],
        [712648., 607541.],
        [593879., 444161.],
        [396583.,  82842.],
        [688076., 833905.],
        [198294., 552987.],
        [561388., 355834.],
        [839757., 190518.],
        [903840., 586229.],
        [409142., 811927.],
        [282066., 560178.],
        [341775., 42

In [37]:
np.shape(ci)

(20, 100, 2)

In [36]:
np.shape(Yis)

(20, 250, 2)

In [53]:
# 随时间变化
for t in range(100):
    mik = [[0 for j in range(k)] for i in range(agent)] # agent*k
    uik = [[np.array([0.,0.]) for j in range(k)] for i in range(agent)] # 2*agent*k
    for i in range(agent):
        last_ci = np.array(ci[i]) # 簇的位置
        for j in range(mi):
            j_dis = np.linalg.norm(last_ci-Yis[i][j],axis = 1)
            j_cluster = np.argmin(j_dis)
            uik[i][j_cluster] += Yis[i][j]
            mik[i][j_cluster] += 1
    uik = np.array(uik)
    x1_x = uik[:,:,0]
    x2_y = uik[:,:,1]
    x3 = np.mat(mik)
####################    
    m_th = 0.1
    ux_th = uy_th = 0.1
    s = 0
    
    while True:
        try:
            A_temp = np.mat(As[s%3]) # 获取通信拓扑结构
            x3 = A_temp*x3
            x1_x = A_temp*x1_x
            x2_y = A_temp*x2_y
            # 判断是否一致
            m_mean = x3.mean(axis = 0)
            m_dis = np.abs(x3 - m_mean).mean()  # m的总差距
            ux_mean = x1_x.mean(axis=0)
            uy_mean = x2_y.mean(axis=0)
            ux_dis = np.abs(x1_x - ux_mean).mean()  # ux的总差距
            uy_dis = np.abs(x2_y - uy_mean).mean()  # uy的总差距
            
            if m_dis <= m_th and ux_dis <=ux_th and uy_dis < uy_th:
                break
        finally:
            s += 1
    x3[x3==0] = 1
    ci_x = (x1_x/x3).A
    ci_y = (x2_y/x3).A
    ci.append(list(zip(ci_x[0],ci_y[0])))